# instalación de paquetes y verificación

In [1]:
# comprobamos que la versión de python sea inferior a 3.8, ya que para usar tensorflow 1.15 se recomienda usar python
# 3.7 o inferior
import sys
import string
python_version=sys.version.split("(")[0]
if int(sys.version.split(".")[1])>7:
    print("you are using a python version higer than 3.7.x, please install python 3.7.x")
else:
    print("congrats, you are using {} which is compatible with this notebook".format(python_version))


congrats, you are using 3.7.9  which is compatible with this notebook


In [15]:
# si todo sale bien,el test debería correr y los resultados aparecerán debajo. Es normal si se salta alguno de los tests 
# que hay
!python3 models/research/object_detection/builders/model_builder_tf1_test.py


Running tests under Python 3.7.9: /home/bigdata/anaconda3/envs/test/bin/python3
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
/home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tensorflow_core/python/training/tracking/data_structures.py:669: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  if not isinstance(wrapped_dict, collections.Mapping):
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params0 (True)
[ RUN      ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[       OK ] ModelBuilderTF1Test.test_create_context_rcnn_from_config_with_params1 (False)
[ RUN      ] ModelBuilderTF1Test.test_create_experimental_model
[       OK ] ModelBuilderTF1Test.test_create_experimental_model
[ RUN      ] ModelBuilderTF1Test.test_create_faster_rcnn_from_config_with_crop_feat

# creación del dataset

In [16]:
# vamos a dividir nuestras imágenes y anotaciones en splits de train, test y validación.
# los argumentos -vr y -tr indican respectivamente los porcentajes de datos reservados para validación y test
!python3 scripts/train_test_split.py -i images_bb -o images_bb -tr 0.1 -vr 0.15 -f JPG
!tree -d images_bb

succesfully created validation split using:  15.0 % of data
succesfully created test split using:  10.0 % of data
succesfully created train split using:  75.0 % of data
images_bb
├── test
├── train
└── val

3 directories


In [21]:
!python3 scripts/json_to_xml.py -i images_bb/train -o images_bb/train
!python3 scripts/json_to_xml.py -i images_bb/val -o images_bb/val
!python3 scripts/json_to_xml.py -i images_bb/test -o images_bb/test

G0011154.xml
G0021262.xml
G0051599.xml
G0061723.xml
G0051637.xml
G0051597.xml
G0031460.xml
G0061737.xml
G0021319.xml
G0061746.xml
G0051527.xml
G0051642.xml
G0011175.xml
G0031435.xml
G0011145.xml
G0051577.xml
G0021268.xml
G0031483.xml
G0011144.xml
G0061816.xml
G0021333.xml
G0051576.xml
G0031476.xml
G0011193.xml
G0011146.xml
G0061798.xml
G0051680.xml
G0021327.xml
G0061736.xml
G0011227.xml
G0061763.xml
G0031438.xml
G0061740.xml
G0011138.xml
G0051655.xml
G0031439.xml
G0011147.xml
G0041505.xml
G0061717.xml
G0021372.xml
G0061811.xml
G0051572.xml
G0031454.xml
G0011150.xml
G0031452.xml
G0031386.xml
G0061838.xml
G0011159.xml
G0011142.xml
G0031431.xml
G0051631.xml
G0051636.xml
G0031442.xml
G0011210.xml
G0011160.xml
G0061755.xml
G0011148.xml
G0021357.xml
G0061720.xml
G0061696.xml
G0021369.xml
G0061745.xml
G0011139.xml
G0021281.xml
G0041503.xml
G0051645.xml
G0021296.xml
G0011156.xml
G0011141.xml
G0061684.xml
G0031449.xml
G0031414.xml
G0061831.xml
G0011149.xml
G0051538.xml
G0011143.xml
G0051546.xml

In [24]:
# ahora tenemos que generar los tfrecords usando estos dataset en formato coco
!python3 scripts/generate_tfrecord.py -x images_bb/train -l training/label_map_bb.pbtxt -o training/train.record
!python3 scripts/generate_tfrecord.py -x images_bb/test -l training/label_map_bb.pbtxt -o training/test.record
!python3 scripts/generate_tfrecord.py -x images_bb/val -l training/label_map_bb.pbtxt -o training/val.record

Successfully created the TFRecord file: training/train.record
Successfully created the TFRecord file: training/test.record
Successfully created the TFRecord file: training/val.record


# Entrenamiento y exportación del modelo

In [26]:
# entrenamos el modelo usando estos tfrecords. El modelo resultante se guardará el carpeta training
!python3 models/research/object_detection/model_main.py --alsologtostderr \
--model_dir=training/ \
--pipeline_config_path=training/faster_rcnn_inception_v2_coco.config \
--num_train_steps=400000

W0909 09:12:11.643194 140046260520768 model_lib.py:771] Forced number of epochs for all eval validations to be 1.
INFO:tensorflow:Maybe overwriting train_steps: 400000
I0909 09:12:11.643353 140046260520768 config_util.py:552] Maybe overwriting train_steps: 400000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0909 09:12:11.643400 140046260520768 config_util.py:552] Maybe overwriting use_bfloat16: False
INFO:tensorflow:Maybe overwriting sample_1_of_n_eval_examples: 1
I0909 09:12:11.643438 140046260520768 config_util.py:552] Maybe overwriting sample_1_of_n_eval_examples: 1
INFO:tensorflow:Maybe overwriting eval_num_epochs: 1
I0909 09:12:11.643476 140046260520768 config_util.py:552] Maybe overwriting eval_num_epochs: 1
W0909 09:12:11.643527 140046260520768 model_lib.py:787] Expected number of evaluation epochs is 1, but instead encountered `eval_on_train_input_config.num_epochs` = 0. Overwriting `num_epochs` to 1.
INFO:tensorflow:create_estimator_and_inputs: use_tpu False, export_

Instructions for updating:
Use `tf.cast` instead.
W0909 09:12:22.938572 140046260520768 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/object_detection/inputs.py:259: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Calling model_fn.
I0909 09:12:25.777052 140046260520768 estimator.py:1148] Calling model_fn.
Instructions for updating:
Please use `layer.__call__` method instead.
W0909 09:12:25.797866 140046260520768 deprecation.py:323] From /home/bigdata/anaconda3/envs/test/lib/python3.7/site-packages/tf_slim/layers/layers.py:2802: Layer.apply (from tensorflow.python.keras.engine.base_layer) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `layer.__call__` method instead.
INFO:tensorflow:Scale of 0 disables regularizer.
I0909 09:12:26.620193 140046260520768 regularizers.py:99] Scale of 0 

INFO:tensorflow:Running local_init_op.
I0909 09:12:34.256578 140046260520768 session_manager.py:500] Running local_init_op.
INFO:tensorflow:Done running local_init_op.
I0909 09:12:34.440859 140046260520768 session_manager.py:502] Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into training/model.ckpt.
I0909 09:12:38.137158 140046260520768 basic_session_run_hooks.py:606] Saving checkpoints for 0 into training/model.ckpt.
2020-09-09 09:12:42.435622: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2020-09-09 09:12:44.097419: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudnn.so.7
2020-09-09 09:12:44.903374: W tensorflow/stream_executor/cuda/ptxas_utils.cc:116] *** WARNING *** You are using ptxas 9.1.108, which is older than 9.2.88. ptxas 9.x before 9.2.88 is known to miscompile XLA code, leading to incorrect results or invalid-address errors

In [ ]:
# aquí nos vamos a fijar en los archivos model.ckpt y vamos a buscar el que tenga el número más alto, ya que este número
# indica el step del proceso de entrenamiento en el que se realizó el checkpoint del modelo
!dir training

In [ ]:
# por si no lo ves claro, corriendo esto te dirá qué debes poner en el script siguiente que generará el grafo de inferencia
import glob
import os
max_step=0
for file in glob.glob("training/*"):
    if ("model.ckpt" in  file):
        step=int (file.split(os.sep)[1].split(".")[1].split("-")[1])
        max_step= step if step>max_step else max_step
print("En el campo trained_checkpoint debes poner lo siguiente: \ntraining/model.ckpt-"+str(max_step))

In [ ]:
# ahora vamos a exportar este modelo como un grafo de inferencia. Este grafo se podrá usar para generar predicciones luego.
# Debemos sustituir los parámetros de entrada ["pipeline_config_path","trained_checkpoint_prefix","output_directory"]
# por los nuestros. Es recomendable elegir un nombre reconocible para la carpeta de salida, como por ejemplo
# "inference_graph" seguido de la fecha y la hora
!python3 models/research/object_detection/export_inference_graph.py --input_type image_tensor \
--pipeline_config_path training/mask_rcnn_inception_v2_coco.config \
--trained_checkpoint_prefix training/model.ckpt-2744\
--output_directory inference_graph_02_09_10_14

In [ ]:
!dir inference_graph_02_09_10_14